In [4]:
using Pkg
Pkg.activate(".")

Activating environment at `~/Documents/Stanford/Stefan/EBCrossFitPaper/Project.toml`


In [18]:
Pkg.add("StatsBase")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 % Resolving package versions...
 Installed BinaryProvider ─ v0.5.7
  Updating `~/Documents/Stanford/Stefan/EBCrossFitPaper/Project.toml`
  [2913bbd2] + StatsBase v0.32.0
  Updating `~/Documents/Stanford/Stefan/EBCrossFitPaper/Manifest.toml`
  [b99e7846] ↑ BinaryProvider v0.5.6 ⇒ v0.5.7


Pkg.status()

In [21]:
using EBayes
using EBayesDatasets
import JuliaDB
using StatsBase

Load in the MovieLens dataset. The preprocessing of the data occurs in the [EBayesDatasets.jl](https://github.com/nignatiadis/EBayesDatasets.jl) package.

In [8]:
mv_lens = EBayesDatasets.MovieLens(;n_min_train=3, prop_test=0.1);

## Pulp Fiction and Urban Justice

In the introduction of the paper we mention the ratings of these two movies, so let us compute these:

In [10]:
movie_titles = JuliaDB.select(mv_lens.movie_df, :title);

In [20]:
pulp_fiction_idx = findall( match.(r"Pulp", movie_titles) .!= nothing)[1]
pulp_fiction = mv_lens.movie_df[pulp_fiction_idx]

(movieId = 296,
 title = "Pulp Fiction (1994)",
 genres = "Comedy|Crime|Drama|Thriller",
 year = 1994,
 test_mean = 4.1734962f0,
 test_n = 60641,
 test_sd = 0.9773011f0,
 train_mean = 4.1809115f0,
 train_n = 6669,
 train_sd = 0.96609414f0,)

In [35]:
# let us average over test and train sets
pulp_fiction_avg = mean( [pulp_fiction[:test_mean], pulp_fiction[:train_mean]],
                    Weights([pulp_fiction[:test_n], pulp_fiction[:train_n]]))
round(Float64(pulp_fiction_avg), digits=1)

4.2

In [39]:
urban_justice_idx = findall( match.(r"Urban Justice", movie_titles) .!= nothing)[1]
urban_justice = mv_lens.movie_df[urban_justice_idx]

(movieId = 66652,
 title = "Urban Justice (2007)",
 genres = "Action",
 year = 2007,
 test_mean = 1.5454545f0,
 test_n = 11,
 test_sd = 0.9862693f0,
 train_mean = 1.8333334f0,
 train_n = 3,
 train_sd = 0.28867513f0,)

In [41]:
# let us average over test and train sets
urban_justice_avg = mean( [urban_justice[:test_mean], urban_justice[:train_mean]],
                    Weights([urban_justice[:test_n], urban_justice[:train_n]]))
round(Float64(urban_justice_avg), digits=1)

1.6

# Set up EBCrossFit with XGBoost

In [50]:
using MLJ

In [52]:
MLJ.@load XGBoostRegressor
xgb_tree = XGBoostRegressor(max_depth=5)
r_num_round = range(xgb_tree, :num_round, lower=2, upper=100)
r_eta = range(xgb_tree, :eta, lower=0.01, upper=1.0)
nested_ranges = [r_num_round, r_eta]
tuned_XGBoost = TunedModel(model=xgb_tree,
                           tuning=Grid(resolution=10),
                           resampling=CV(nfolds=5),
                           ranges=nested_ranges,
                           measure=rms)l

┌ Info: A model type "XGBoostRegressor" is already loaded. 
│ No new code loaded. 
└ @ MLJModels /Users/ignatiad/.julia/packages/MLJModels/SD4Ue/src/loading.jl:41


UndefVarError: UndefVarError: l not defined

# Evaluation of methods

In [43]:
ss = mv_lens.Zs_train  #Z_i and σ_i calculated based on 10% of users
ground_truth = mv_lens.Zs_test.Z; #average movie ratings based on other 90% of users

In [45]:
ss

12481-element StructArray(::Array{Float64,1}, ::Array{Float64,1}) with eltype NormalSample{Float64}:
 NormalSample{Float64}(3.915266275405884, 0.01341832039868122)  
 NormalSample{Float64}(3.2022109031677246, 0.0201176920943941)  
 NormalSample{Float64}(3.15059757232666, 0.02645977858585608)   
 NormalSample{Float64}(2.8026819229125977, 0.058021348063866655)
 NormalSample{Float64}(3.091531276702881, 0.027415793044111868) 
 NormalSample{Float64}(3.8189802169799805, 0.019402497795562867)
 NormalSample{Float64}(3.3526148796081543, 0.026386293639319304)
 NormalSample{Float64}(3.0565216541290283, 0.08740960314020435) 
 NormalSample{Float64}(3.0024330615997314, 0.046236734874957754)
 NormalSample{Float64}(3.431675910949707, 0.017478881086919393) 
 NormalSample{Float64}(3.651515245437622, 0.021805049915141168) 
 NormalSample{Float64}(2.6008522510528564, 0.04996164588421241) 
 NormalSample{Float64}(3.316793918609619, 0.08189783492788894)  
 ⋮                                                    

### Apply unbiased predictor (MLE)

In [58]:
unbiased_preds = ss.Z
unbiased_preds_errors = (unbiased_preds .- ground_truth).^2;

In [59]:
unbiased_mse = mean(unbiased_preds_errors)
# unbiased_mse_std = std(mle_errors)/sqrt(length(ss))


0.09830188086050119

### Apply SURE predictor

In [57]:
sure_fit = fit(Normal(), SURE(), GrandMeanLocation(), ss)
sure_preds = predict(sure_fit);
sure_preds_errors = (sure_preds .- ground_truth).^2;

In [60]:
sure_mse = mean(sure_preds_errors)
# sure_mse_std = std(sure_errors)/sqrt(length(ss))
#

0.0609882885478651

In [ ]:
# ebcf_mse = mean(ebcf_errors)
# ebcf_mse_std = std(ebcf_errors)/sqrt(length(ss))

In [ ]:
sure_preds_errors = ss.Z
